# Individual Model Generation

Individual models are generated in this file. These models use 80% of a subject's data for training and 20% for testing.

The models are saved in a CSV file called `train_individual.csv` for the training models and `test_individual.csv` for the testing models. The feature importance data is also saved for each model. This data is saved in a CSV file called `feature_importance_individual.csv`. These files are in the directory `outputs/individual/`.

In [1]:
from pandas import read_csv
from pycaret.regression import *
import csv
import pandas as pd

In [2]:
%%capture
df = read_csv('data/st-final_data.csv')
if 'Unnamed: 0' in df.columns:
    df = df.drop('Unnamed: 0', axis=1)

In [3]:
def split_fourths(group):
    n = len(group)
    index_25 = int(n * 0.25)
    index_50 = int(n * 0.50)
    index_75 = int(n * 0.75)
    train_segments = [group.iloc[0:index_25], group.iloc[index_50:index_75]]
    test_segments = [group.iloc[index_25:index_50], group.iloc[index_75:n]]
    return pd.concat(train_segments, ignore_index=True), pd.concat(test_segments, ignore_index=True)

In [7]:
train_individual_df = pd.DataFrame(columns=["Model", "MAE", "MSE", "RMSE", "R2", "RMSLE", "MAPE", "TT (Sec)", "Subject"])
test_individual_df = pd.DataFrame(columns=["Model", "MAE", "MSE", "RMSE", "R2", "RMSLE", "MAPE", "Subject"])
feature_importance_individual_df = pd.DataFrame(columns=["Feature", "Value", "Model", "Subject"])


# Predictors to remove: experimental_condition, subject
for subject in df['subject'].unique():  # df['subject'].unique()
    print(subject)
    # New way!
    df_subject = df[df['subject'] == subject]
    
    train_frames = []
    test_frames = []

    for _, group in df_subject.groupby('task'):
        train_segment, test_segment = split_fourths(group)
        train_frames.append(train_segment)
        test_frames.append(test_segment)

    train = pd.concat(train_frames, ignore_index=True)
    test = pd.concat(test_frames, ignore_index=True)
    
    print(len(train.index), len(test.index))

    reg = setup(
        data = train, test_data = test,
        target='RPE', index = False,
        ignore_features=['sub_tsk', 'task', 'time', 'time_pc', 'time_perc', 'subject'],
        session_id = 2024)

    best = compare_models(sort='MAE', n_select = 18)
    all = pull()
    all['Subject'] = str(subject)

    train_individual_df = pd.concat([train_individual_df, all], ignore_index=True)

    test_results = pd.DataFrame()
    for model in best:
        test_result = predict_model(model, verbose = False)
        test_result = pull()
        test_results = test_results.append(test_result)

        try:
            importance = pd.DataFrame({'Feature': get_config('X_train').columns, 'Value' : abs(model.feature_importances_)}).sort_values(by='Value', ascending=False).reset_index().drop('index', axis=1)
        except:
            try:
                importance = pd.DataFrame({'Feature': get_config('X_train').columns, 'Value' : abs(model.coef_)}).sort_values(by='Value', ascending=False).reset_index().drop('index', axis=1)
            except:
                importance = pd.DataFrame({'Feature': ['error'], 'Value': [0]})
        importance['Model'] = str(model)
        importance['Subject'] = str(subject)

        feature_importance_individual_df = pd.concat([feature_importance_individual_df, importance], ignore_index=True)


    test_results['Subject'] = str(subject)
    test_individual_df = pd.concat([test_individual_df, test_results], ignore_index=True)


train_individual_df.to_csv('outputs/individual/train_individual.csv', header=True, index=False)
test_individual_df.to_csv('outputs/individual/test_individual.csv', header=True, index=False)
feature_importance_individual_df.to_csv('outputs/individual/feature_importance_individual.csv', header=True, index=False)

Subject 1
4 6


,Description,Value
0,Session id,2024
1,Target,RPE
2,Target type,Regression
3,Original data shape,"(10, 40)"
4,Transformed data shape,"(10, 34)"
5,Transformed train set shape,"(4, 34)"
6,Transformed test set shape,"(6, 34)"
7,Ignore features,6
8,Numeric features,33
9,Preprocess,True


Subject 2
8 12


,Description,Value
0,Session id,2024
1,Target,RPE
2,Target type,Regression
3,Original data shape,"(20, 40)"
4,Transformed data shape,"(20, 34)"
5,Transformed train set shape,"(8, 34)"
6,Transformed test set shape,"(12, 34)"
7,Ignore features,6
8,Numeric features,33
9,Preprocess,True


Subject 3
8 12


...

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
dt,Decision Tree Regressor,1.2500,2.9500,1.3121,nan,0.0950,0.0989,0.1460
gbr,Gradient Boosting Regressor,1.4154,3.5665,1.4590,nan,0.1145,0.1192,0.1560
et,Extra Trees Regressor,1.6675,4.4037,1.7138,nan,0.1322,0.1427,0.1500
ada,AdaBoost Regressor,1.7500,6.5500,1.8536,nan,0.1365,0.1349,0.1580
ridge,Ridge Regression,2.1565,6.2036,2.1854,nan,0.1817,0.2040,0.1600
rf,Random Forest Regressor,2.1625,5.4347,2.1933,nan,0.1845,0.2143,0.1480
br,Bayesian Ridge,2.2444,6.9096,2.2756,nan,0.1797,0.2002,0.1800
huber,Huber Regressor,2.4227,6.8838,2.4438,nan,0.2388,0.2627,0.1770
en,Elastic Net,2.4448,7.3473,2.4647,nan,0.2031,0.2278,0.1910
llar,Lasso Least Angle Regression,2.5092,7.5957,2.5236,nan,0.2125,0.2413,0.1480


Subject 29
4 6


,Description,Value
0,Session id,2024
1,Target,RPE
2,Target type,Regression
3,Original data shape,"(10, 40)"
4,Transformed data shape,"(10, 34)"
5,Transformed train set shape,"(4, 34)"
6,Transformed test set shape,"(6, 34)"
7,Ignore features,6
8,Numeric features,33
9,Preprocess,True


Subject 33
8 12


,Description,Value
0,Session id,2024
1,Target,RPE
2,Target type,Regression
3,Original data shape,"(20, 40)"
4,Transformed data shape,"(20, 34)"
5,Transformed train set shape,"(8, 34)"
6,Transformed test set shape,"(12, 34)"
7,Ignore features,6
8,Numeric features,33
9,Preprocess,True
